Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Install and Import Libraries
Let's install PySpark:

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

## Build Spark Session

In [2]:
spark=SparkSession.builder.getOrCreate()

## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [3]:
train=spark.read.csv('train.csv',header= True,inferSchema=True)
test=spark.read.csv('test.csv',header= True,inferSchema=True)

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [5]:
type(train)

pyspark.sql.dataframe.DataFrame

**Show 5 rows.**

In [8]:
train.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

**Display schema for the dataset:**

In [9]:
train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [11]:
train.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [12]:
train.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [21]:
survived=train.select(F.col('Survived')).where('Survived==0').count()
not_survived=train.select(F.col('Survived')).where('Survived==1').count()

**Display your result:**

In [22]:
print('number of survived= ',survived)
print('number of didnt survived= ',not_survived)

number of survived=  549
number of didnt survived=  342


**Can you display your answer in ratio form?(Hint: Use "UDF" Function. (Hint: Use "UDF" Function. This is a hint you can use any method.)**






In [34]:
print('number of survived ratio= ',(survived/(survived+not_survived))*100)
print('number of didnt survived ratio= ',(not_survived/(survived+not_survived))*100)

number of survived ratio=  61.61616161616161
number of didnt survived ratio=  38.38383838383838


**Can you get the number of males and females?**


In [35]:
male=train.select(F.col('Sex')).filter("Sex=='male'").count()
female=train.select(F.col('Sex')).filter("Sex=='female'").count()
print('male= ',male, ' female= ',female)

male=  577  female=  314


**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column. This is a hint you can use any method.)

In [13]:
train.select('sex','Survived').groupBy('Sex').avg().show()
train.select('sex','Survived').where('Survived==1').groupBy('Sex').count().show()


+------+-------------------+
|   Sex|      avg(Survived)|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+

+------+-----+
|   Sex|count|
+------+-----+
|female|  233|
|  male|  109|
+------+-----+



**Create temporary view PySpark:**

In [4]:
train.createOrReplaceTempView('train_temp')

**How many people survived, and how many didn't survive? By SQL:**

In [63]:
spark.sql('select survived,count(survived) as count_survived from train_temp group by(survived)').show()

+--------+--------------+
|survived|count_survived|
+--------+--------------+
|       1|           342|
|       0|           549|
+--------+--------------+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column. This is a hint you can use any method.)

**Can you do this via SQL?**

In [11]:
spark.sql('select sex, avg(survived) from train_temp group by (sex)').show()

+------+-------------------+
|   sex|      avg(survived)|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+



**Display a ratio for "p-class": SUM(Survived)/count for p-class**


In [25]:
spark.sql('select Pclass,(sum(survived)/count(Pclass)) as pclass_ratio from train_temp group by (Pclass)').show()

+------+-------------------+
|Pclass|       pclass_ratio|
+------+-------------------+
|     1| 0.6296296296296297|
|     3|0.24236252545824846|
|     2|0.47282608695652173|
+------+-------------------+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [27]:
data=train.union(test)
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

**Display count:**

In [28]:
data.count()

1329

**Can you define the number of null values in each column?**


In [41]:
data.na.df.count()

1329

**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [47]:
data.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



## Preprocessing 

**Create Temporary view PySpark:**

In [48]:
train.createOrReplaceTempView('train_temp')

**Can you show the "name" column from your temporary table?**

In [50]:
spark.sql('select name from train_temp').show(truncate=False)

+-------------------------------------------------------+
|name                                                   |
+-------------------------------------------------------+
|Braund, Mr. Owen Harris                                |
|Cumings, Mrs. John Bradley (Florence Briggs Thayer)    |
|Heikkinen, Miss. Laina                                 |
|Futrelle, Mrs. Jacques Heath (Lily May Peel)           |
|Allen, Mr. William Henry                               |
|Moran, Mr. James                                       |
|McCarthy, Mr. Timothy J                                |
|Palsson, Master. Gosta Leonard                         |
|Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)      |
|Nasser, Mrs. Nicholas (Adele Achem)                    |
|Sandstrom, Miss. Marguerite Rut                        |
|Bonnell, Miss. Elizabeth                               |
|Saundercock, Mr. William Henry                         |
|Andersson, Mr. Anders Johan                            |
|Vestrom, Miss

**Run this code:**

In [54]:
import pyspark.sql.functions as F
combined = data.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
combined.createOrReplaceTempView('combined')

**Display "Title" column and count "Title" column:**

In [56]:
spark.sql('select Title from combined').show()

+------+
| Title|
+------+
|    Mr|
|   Mrs|
|  Miss|
|   Mrs|
|    Mr|
|    Mr|
|    Mr|
|Master|
|   Mrs|
|   Mrs|
|  Miss|
|  Miss|
|    Mr|
|    Mr|
|  Miss|
|   Mrs|
|Master|
|    Mr|
|   Mrs|
|   Mrs|
+------+
only showing top 20 rows



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [64]:
title_count=spark.sql('select Title,count(title) as countTitle from combined group by(title) ORDER by countTitle')
title_count.show()

+--------+----------+
|   Title|countTitle|
+--------+----------+
|     Mme|         1|
|      Ms|         1|
|     Don|         1|
|Jonkheer|         2|
|    Capt|         2|
|    Lady|         2|
|     Sir|         2|
|Countess|         2|
|   Major|         3|
|     Col|         4|
|    Mlle|         4|
|     Rev|         9|
|      Dr|        11|
|  Master|        56|
|     Mrs|       186|
|    Miss|       257|
|      Mr|       786|
+--------+----------+



In [83]:
list_titles=list(map(lambda row: row.asDict(), title_count.collect()))
titles_map={row['Title']:'rare' if row['countTitle']<56 else row['Title'] for row in list_titles }
titles_map

{'Don': 'rare',
 'Mme': 'rare',
 'Ms': 'rare',
 'Countess': 'rare',
 'Lady': 'rare',
 'Capt': 'rare',
 'Sir': 'rare',
 'Jonkheer': 'rare',
 'Major': 'rare',
 'Col': 'rare',
 'Mlle': 'rare',
 'Rev': 'rare',
 'Dr': 'rare',
 'Master': 'Master',
 'Mrs': 'Mrs',
 'Miss': 'Miss',
 'Mr': 'Mr'}

**Run the function:**

In [84]:
def impute_title(title):
    return titles_map[title]# Title_map is your dictionary. please change this name with your dictionary name.

**Apply the function on "Title" column using UDF:**

In [89]:
convertUDF = F.udf(lambda z: impute_title(z))
converted_df=combined.select(convertUDF(F.col('title')).alias('title'))
converted_df.show()

+------+
| title|
+------+
|    Mr|
|   Mrs|
|  Miss|
|   Mrs|
|    Mr|
|    Mr|
|    Mr|
|Master|
|   Mrs|
|   Mrs|
|  Miss|
|  Miss|
|    Mr|
|    Mr|
|  Miss|
|   Mrs|
|Master|
|    Mr|
|   Mrs|
|   Mrs|
+------+
only showing top 20 rows



**Display "Title" from table and group by "Title" column:**

In [92]:
converted_df.select('title').groupBy('title').count().show()

+------+-----+
| title|count|
+------+-----+
|  rare|   44|
|  Miss|  257|
|Master|   56|
|    Mr|  786|
|   Mrs|  186|
+------+-----+



## **Preprocessing Age**

**Based on the "age" column mean, you will fill in the missing age values:**

In [102]:
age_mean=combined.select(F.avg('age')).collect()[0][0]

**Fill missing with "age" mean:**

In [105]:
combined=combined.na.fill(age_mean,subset=['age'])

## **Preprocessing Embarked**

**Select "Embarked" column, count them, order by count Desc, and save in grouped_Embarked variable:**




In [109]:
grouped_Embarked=combined.select('embarked').groupBy('embarked').count()

**Show "groupped_Embarked" your variable:**

In [110]:
grouped_Embarked.show()

+--------+-----+
|embarked|count|
+--------+-----+
|       Q|  111|
|    null|    3|
|       C|  253|
|       S|  962|
+--------+-----+



**Get max of groupped_Embarked:** 

In [120]:
grouped_Embarked.select(F.max('count')).show()

+----------+
|max(count)|
+----------+
|       962|
+----------+



**Fill missing values with max 'S' of grouped_Embarked:**

In [130]:
grouped_Embarked=combined.na.fill('S',subset=['embarked'])
grouped_Embarked.show()

+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|              35.0|    1|    0|          113803|   53.1| C123|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia..

## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



In [137]:
firstChar = F.udf(lambda word:word[0])
grouped_Embarked=grouped_Embarked.withColumn('cabin',firstChar(F.col('cabin')))

**Show the result:**

In [146]:
grouped_Embarked.show()

+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25|    U|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|    C|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925|    U|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|              35.0|    1|    0|          113803|   53.1|    C|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia..

In [145]:
grouped_Embarked.take(1)

[Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, cabin='U', Embarked='S', Title='Mr')]

**Create the temporary view:**

In [139]:
grouped_Embarked.createTempView('temb2')

**Select "Cabin" column, count "Cabin" column, Group by "Cabin" column, Order By count DESC**  

In [141]:
spark.sql('select Cabin, count(Cabin) as count from temb2 group by (cabin) order by count DESC').show()

+-----+-----+
|Cabin|count|
+-----+-----+
|    U| 1021|
|    C|   82|
|    B|   77|
|    D|   52|
|    E|   51|
|    A|   23|
|    F|   18|
|    G|    4|
|    T|    1|
+-----+-----+



**Fill missing values with "U":**

In [131]:
grouped_Embarked=grouped_Embarked.na.fill('U',subset=['cabin'])

**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

In [148]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorAssembler
stringIndexer= StringIndexer(inputCols=['Sex','Ticket','cabin','Embarked','Title'],outputCols=['Sex_index','Ticket_index','cabin_index','Embarked_index','Title_index'],handleInvalid='skip')


**OneHotEncoder(inputCols=None, outputCols=None)**

A one-hot encoder that maps a column of category indices to a column of binary vectors, with at most a single one-value per row that indicates the input category index. For example with 5 categories, an input value of 2.0 would map to an output vector of [0.0, 0.0, 1.0, 0.0]. The last category is not included by default (configurable via dropLast), because it makes the vector entries sum up to one, and hence linearly dependent. So an input value of 4.0 maps to [0.0, 0.0, 0.0, 0.0].

In [149]:
one_hot_encoder= OneHotEncoder(inputCols=['Sex_index','Ticket_index','cabin_index','Embarked_index','Title_index'],outputCols=['Sex_ohe','Ticket_ohe','cabin_ohe','Embarked_ohe','Title_ohe'])

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None). A feature transformer that merges multiple columns into a vector column.**



In [151]:
assemblerInputs=['Sex_ohe','Ticket_ohe','cabin_ohe','Embarked_ohe','Title_ohe','Pclass','Age','SibSp','Parch','Fare']

vecAssembler= VectorAssembler(inputCols=assemblerInputs,outputCol='features')

**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [153]:
x_train, X_test = grouped_Embarked.randomSplit([.8,.2],seed=42)

**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [161]:
from pyspark.ml.classification import RandomForestClassifier 
from pyspark.ml.pipeline import Pipeline

rf=RandomForestClassifier(featuresCol='features',labelCol='Survived')

pipline=Pipeline(stages=[stringIndexer,one_hot_encoder,vecAssembler,rf])
piplineModel=pipline.fit(x_train)

In [162]:
predDF = piplineModel.transform(X_test)

**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [173]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluatore=MulticlassClassificationEvaluator(labelCol='Survived',predictionCol='prediction',metricName='accuracy')
acc=evaluatore.evaluate(predDF)
acc

0.7865168539325843

**When you are finished send the project via Google classroom**
**Please let me know if you have any questions.**
* nabieh.mostafa@yahoo.com
* +201015197566 (Whatsapp)

**Don't Hate me, I push you to learn**

**I will help you to become an awesome data engineer.**

**Why did I say that "Data Engineer"?**

**Tricky question, but an optional question, if you would like to know the answer, ask me.**
